# TF-IDF（Term frequency-inverse document frequency ）

Term Frequency-Inverse Document Frequency, commonly known as TF-IDF, is a numerical statistic that reflects the importance of a word in a document relative to a collection of documents (corpus). It is widely used in natural language processing and information retrieval to evaluate the significance of a term within a specific document in a larger corpus.

In this part ，we are going to learn how to use TF_IDF model to represent tests as vectors.

## Zipf’s Law

First, let’s notice that words are typically distributed in languages according to a power law called Zipf’s law. Here’s how Wikipedia describes it.

> Zipf’s law was originally formulated in terms of quantitative linguistics, stating that given some corpus of natural language utterances, the frequency of any word is inversely proportional to its rank in the frequency table. […] For example, in the Brown Corpus of American English text, the word “the” is the most frequently occurring word, and by itself accounts for nearly 7% of all word occurrences. True to Zipf’s Law, the second-place word “of” accounts for slightly over 3.5% of words […]. Only 135 vocabulary items are needed to account for half the Brown Corpus.

So, this is what the distribution of the frequencies of the words in a corpus should look like.

<center><img src="https://static-1300131294.cos.ap-shanghai.myqcloud.com/images/deep-learning/NLP/zipf.png" width="80%" class="bg-white mb-1" ><br> Image: Zipf</center>

Let’s quickly verify if this distribution holds also for the dataset of Medium articles that we used in the last projects.

## Zipf’s Law in Medium Articles

Let’s import the necessary libraries. The only different library from the ones in the previous lessons is **plotly**, which is a data visualization library for making beautiful interactive plots.

In [ ]:
%pip install plotly
from huggingface_hub import hf_hub_download
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import pandas as pd
import plotly.express as px

from collections import Counter
import re

We then download the dataset of Medium articles from the Hugging Face Hub and convert it to a `dataframe.pandas`

In [3]:
# download dataset of Medium articles from 
# https://huggingface.co/datasets/fabiochiu/medium-articles
df_articles = pd.read_csv(
  hf_hub_download("fabiochiu/medium-articles", repo_type="dataset", filename="medium_articles.csv")
)

# There are 192,368 articles in total, but let's sample only 30,000 of them to
# make computations faster
df_articles = df_articles.sample(n=30000)

df_articles.head()

,title,text,url,authors,timestamp,tags
60231,Are we Turning the Internet Against Ourselves?,Are we Turning the Internet Against Ourselves?...,https://asingularstory.medium.com/are-we-turni...,['A Singular Story'],2020-02-19 16:51:16.569000+00:00,"['Future', 'Society', 'Philosophy', 'Politics'..."
15966,Blockport Community Update: June,"In this article, a brief overview is provided ...",https://medium.com/blockport/blockport-communi...,['Sebastiaan Lichter'],2018-06-15 13:49:24.144000+00:00,"['Trading', 'Bitcoin', 'Community', 'Ethereum'..."
40399,"Google tweaks algorithm, world ends: The one t...",How You Could Have Prevented This\n\nI worked ...,https://medium.com/the-redesign/google-tweaks-...,['Angela Obias-Tuban'],2016-09-05 12:03:22.482000+00:00,"['UX Research', 'Search Engine Marketing', 'Go..."
1591,The Future of Love: Robot Sex and AI Relations...,"June 6th, 2048 — after working out for ten min...",https://orge.medium.com/the-future-of-love-rob...,['Orge Castellano'],2018-05-29 14:42:46.515000+00:00,"['Love', 'Sex', 'Artificial Intelligence', 'Fu..."
173909,Ad Click Prediction,Ad Click Prediction\n\nTo predict whether cust...,https://medium.com/@ghoshratul063/ad-click-pre...,['Ratul Ghosh'],2021-07-06 18:59:07.664000+00:00,"['Machine Learning', 'Predictive Analytics', '..."


Next, let’s do some simple text preprocessing and count how many occurrences we have for each token in the whole dataset.

In [4]:
def from_text_to_counter(full_text):
  # convert texts to lowercase
  full_text_lower = full_text.lower()

  # remove punctuation from articles
  full_text_no_punctuation = re.sub(r'[^\w\s]', ' ', full_text_lower)

  # tokenize texts
  full_text_tokenized = word_tokenize(full_text_no_punctuation)

  # count the occurrences of each token
  token_counter = Counter(full_text_tokenized)

  return token_counter

# count the occurrences of each token
full_text = " ".join(df_articles["text"].values)
token_counter = from_text_to_counter(full_text)

We are now ready to plot the number of occurrences of the most frequent tokens in the dataset.

In [5]:
# get occurrences of top 30 tokens
top_n = 30
xs, ys = list(zip(*token_counter.most_common(top_n)))

# plot
plot_title = "Tokens and their occurrencies on the Medium dataset"
labels = {
  "x": "Tokens",
  "y": "Number of occurrencies"
}
fig = px.bar(x=xs, y=ys, template="plotly_dark", title=plot_title, labels=labels)
fig.show()

As expected, the most common tokens are prepositions and articles such as “the”, “to”, and “and”. Let’s see if their occurrences are approximately inversely proportional to their rank.

In [6]:
# get occurrences of top 30 tokens
top_n = 30
xs, ys = list(zip(*token_counter.most_common(top_n)))

# get zipf values of top 30 occurrences
ys_zipfs = [ys[0]]
for i in range(1, len(ys)):
  new_value = (ys_zipfs[0] / (i + 1))
  ys_zipfs.append(new_value)

# plot
plot_title = "Tokens and their occurrencies on the Medium dataset (red) VS Zipf's law (blue)"
labels = {
  "x": "Tokens",
  "y": "Number of occurrencies"
}
fig = px.line(x=xs, y=ys_zipfs, template="plotly_dark", title=plot_title, labels=labels)
fig.add_bar(x=xs, y=ys)
fig.update_layout(showlegend=False)
fig.show()

It looks like there are some differences between the values predicted by the Zipf’s law and the number of occurrences in the Medium dataset, maybe because articles on Medium don’t properly represent common English usage such as in the Brown Corpus.

## Zipf’s Law in Brown Corpus

Let’s try doing the same with the texts available in from the Brown Corpus`.nltk`

In [11]:
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\zhongmeiqi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


True

We then download all the corpora and count the occurrences of each token.

In [12]:
# list of Shakespeare corpora
file_ids = nltk.corpus.gutenberg.fileids()

# get all corpora
corpora = [
    " ".join(nltk.corpus.gutenberg.words(corpus_name))
    for corpus_name in file_ids
]

# concatenate all corpora
full_text = " ".join(corpora)

# count the occurrences of each token
token_counter = from_text_to_counter(full_text)

Let’s plot the number of occurrences of the most frequent tokens.

In [13]:
# get occurrences of top 30 tokens
top_n = 30
xs, ys = list(zip(*token_counter.most_common(top_n)))

# get zipf values of top 30 occurrences
ys_zipfs = [ys[0]]
for i in range(1, len(ys)):
  new_value = (ys_zipfs[0] / (i + 1))
  ys_zipfs.append(new_value)

# plot
plot_title = "Tokens and their occurrencies on the Brown corpora (red) VS Zipf's law (blue)"
labels = {
  "x": "Tokens",
  "y": "Number of occurrencies"
}
fig = px.line(x=xs, y=ys_zipfs, template="plotly_dark", title=plot_title, labels=labels)
fig.add_bar(x=xs, y=ys)
fig.update_layout(showlegend=False)
fig.show()

The distribution is indeed more similar to the one suggested by the Zipf’s law.

## Zipf’s law and Stopwords

So, Zipf’s law gives us an intuition of how tokens are typically distributed in texts. Notice that the tokens with the most occurrences are basically the stopwords that we used in the previous lessons.

In [14]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhongmeiqi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
english_stopwords = stopwords.words('english')
print(english_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

What if, when vectorizing texts for our search engine example, we assign a weight to each token that is inversely proportional to its frequency in the texts (similarly to its probability under the Zipf’s distribution)? In this way, rare words are given a higher weight than more common words. The intuition is that a document is better described by the rare words it contains, rather than by its common words.

TF-IDF is a heuristic that works according to this intuition.

## TF-IDF

[TF-IDF (Term Frequency-Inverse Document Frequency)](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) is a way of measuring how relevant a word is (by giving it a score) to a document in a collection of documents.

This is done by multiplying two scores:
- Term Frequency (TF): How many times a word appears in a document.
- Inverse Document Frequency (IDF): The inverse document frequency of the word across a collection of documents. Rare words have high scores, common words have low scores (similar to the intuition we mentioned before with the Zipf’s law).

So, this is how the TF-IDF score of a word relative to a document is computed.

> TF-IDF(word, document) = TF(word, document) * IDF(word)

The specific implementation behind TF and IDF may vary, but the important thing is that TF is somewhat proportional to the number of occurrences of that word in the document, and IDF is somewhat inversely proportional to the number of occurrences of that word in all the documents in the corpus.

TF-IDF has many uses, such as in information retrieval, text analysis, keyword extraction, and as a way of obtaining numeric features from text for machine learning algorithms.

### TF-IDF Example

Suppose we are looking for documents using the query Q and our database is composed of the documents D1, D2, and D3.
- Q: The cat.
- D1: The cat is on the mat.
- D2: My dog and cat are the best.
- D3: The locals are playing.

There are several ways of calculating TF, with the simplest being a raw count of instances a word appears in a document. We’ll compute the TF scores using the ratio of the count of instances over the length of the document.

> TF(word, document) = “number of occurrences of the word in the document” / “number of words in the document”

Let’s compute the TF scores of the words “the” and “cat” (i.e. the query words) with respect to the documents D1, D2, and D3.

> TF(“the”, D1) = 2/6 = 0.33

> TF(“the”, D2) = 1/7 = 0.14

> TF(“the”, D3) = 1/4 = 0.25

> TF(“cat”, D1) = 1/6 = 0.17

> TF(“cat”, D2) = 1/7 = 0.14

> TF(“cat”, D3) = 0/4 = 0

IDF can be calculated by taking the total number of documents, dividing it by the number of documents that contain a word, and calculating the logarithm. If the word is very common and appears in many documents, this number will approach 0. Otherwise, it will approach 1.

> IDF(word) = log(number of documents / number of documents that contain the word)

Let’s compute the IDF scores of the words “the” and “cat”.
> IDF(“the”) = log(3/3) = log(1) = 0

> IDF(“cat”) = log(3/2) = 0.18

Multiplying TF and IDF gives the TF-IDF score of a word in a document. The higher the score, the more relevant that word is in that particular document.

> TF-IDF(word, document) = TF(word, document) * IDF(word)

Let’s compute the TF-IDF scores of the words “the” and “cat”.

> TF-IDF(“the”, D1) = 0.33 * 0 = 0

> TF-IDF(“the, D2) = 0.14 * 0 = 0

> TF-IDF(“the”, D3) = 0.25 * 0 = 0

> TF-IDF(“cat”, D1) = 0.17 * 0.18= 0.0306

> TF-IDF(“cat, D2) = 0.14 * 0.18= 0.0252

> TF-IDF(“cat”, D3) = 0 * 0 = 0

The next step is to use a ranking function to order the documents according to the TF-IDF scores of their words. We can use the average TF-IDF word scores over each document to get the ranking of D1, D2, and D3 with respect to the query Q.

> Average TF-IDF of D1 = (0 + 0.0306) / 2 = 0.0153

> Average TF-IDF of D2 = (0 + 0.0252) / 2 = 0.0126

> Average TF-IDF of D3 = (0 + 0) / 2 = 0

Looks like the word “the” does not contribute to the TF-IDF scores of each document. This is because “the” appears in all of the documents and thus it is considered a not-relevant word.

As a conclusion, when performing the query “The cat” over the collection of documents D1, D2, and D3, the ranked results would be:
- D1: The cat is on the mat.
- D2: My dog and cat are the best.
- D3: The locals are playing.


### TF-IDF is a Heuristic

Keep in mind that TF-IDF is a heuristic and has several formulations, such as [Okapi BM25](https://en.wikipedia.org/wiki/Okapi_BM25). There’s no formulation that is always better than others in every use case. However, all these formulations have in common that rare words should have a higher importance than recurrent words.

## Your turn! 🚀

Assignment - tbd

# Acknowledgments

Thanks to [NLPlanate](https://www.nlplanet.org) for creating [Representing Texts as Vectors: TF-IDF](https://www.nlplanet.org/course-practical-nlp/01-intro-to-nlp/09-text-as-vectors-bow-tfidf#tf-idf-is-a-heuristic). It inspires the majority of the content in this chapter.